# Exercise 3 analysis

Tianfang Wang a1788845

## Loading modules

In [81]:
import os
import re
# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git
from git import RemoteProgress
from git import Repo


## Case 1 CVE-2015-5174

In [82]:
repo1 = Repo("~/Documents/tomcat80")
fixing_commit_1 = "d0e4c578f6"


## Q(a) What was the message and title of the fixing commit? Was there any mention of fixing a bug or vulnerability? 

In [83]:
res1 = repo1.git.log(-1, '--pretty=%B', fixing_commit_1)

print("Title:",res1)

Title: Refactor to remove the circular dependency between o.a.catalina and
o.a.naming

git-svn-id: https://svn.apache.org/repos/asf/tomcat/trunk@1239048 13f79535-47bb-0310-9956-ffa450edef68



## Q(b) How many total files were affected in the fixing commit? 

In [84]:
res2 = repo1.git.diff('--name-only', fixing_commit_1, fixing_commit_1 + '^').splitlines()
print("Number of affected files: %d"%len(res2))

Number of affected files: 10


## Q(c) How many total directories were affected in the fixing commit? For example, if a file path is: abc/def/File.java, then its directory is abc/def.

In [85]:
res3 = repo1.git.diff('--dirstat', fixing_commit_1, fixing_commit_1 + '^').splitlines()
print("Number of affected directories: %d"%len(res3))

Number of affected directories: 6


## Q(d) How many total lines of code (including comments and blank lines) were deleted? 

In [86]:
res4 = repo1.git.show(fixing_commit_1).splitlines()
res4 = [line for line in res4 if re.compile('^-$|^-[^-]').match(line)]
print("Number of deleted lines (including comments and blank lines): %d"%len(res4))

Number of deleted lines (including comments and blank lines): 71


## Q(e) How many total lines of code (including comments and blank lines) were added?

In [87]:
res5 = repo1.git.show(fixing_commit_1).splitlines()
res5 = [line for line in res5 if re.compile('^\+$|^\+[^\+]').match(line)]
print("Number of added lines (including comments and blank lines): %d"%len(res5))

Number of added lines (including comments and blank lines): 157


## Q(f) How many total lines of code (excluding comments and blank lines) were deleted? 

In [88]:
res6 = repo1.git.show('--ignore-blank-lines', fixing_commit_1).splitlines()
res6 = [line for line in res6 if re.compile('^-[^-]').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*/').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res6))

Number of added lines (excluding comments and blank lines): 49


## Q(g) How many total lines of code (excluding comments and blank lines) were added? 

In [89]:
res7 = repo1.git.show('--ignore-blank-lines', fixing_commit_1).splitlines()
res7 = [line for line in res7 if re.compile('^\+[^\+]').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*/').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res7))

Number of added lines (excluding comments and blank lines): 69


## Q(h) How many days were between the current fixing commit and the previous commit of each affected file?

In [90]:
for file in repo1.git.diff('--name-only', fixing_commit_1, fixing_commit_1 + '^').splitlines():
    res8 = repo1.git.log(-2, '--pretty=%ct', file).splitlines();
    if len(res8) == 2:
        print(" For %s , the days between the current fixing commit and the previous commit is %.1f days"%(file, (int(res8[0])-int(res8[1]))/86400))
    else:
        print(" %s:  N/A This file doesn't have previous commit"%file)

 For java/org/apache/catalina/core/ApplicationContext.java , the days between the current fixing commit and the previous commit is 79.5 days
 For java/org/apache/catalina/servlets/WebdavServlet.java , the days between the current fixing commit and the previous commit is 12.8 days
 For java/org/apache/catalina/ssi/SSIServletExternalResolver.java , the days between the current fixing commit and the previous commit is 101.6 days
 For java/org/apache/catalina/ssi/SSIServletRequestUtil.java , the days between the current fixing commit and the previous commit is 101.6 days
 For java/org/apache/catalina/util/RequestUtil.java , the days between the current fixing commit and the previous commit is 12.8 days
 For java/org/apache/naming/resources/FileDirContext.java , the days between the current fixing commit and the previous commit is 33.6 days
 java/org/apache/tomcat/util/http/RequestUtil.java:  N/A This file doesn't have previous commit
 For res/checkstyle/org-import-control.xml , the days be

## Q(i) How many time has each affected file of the current fixing commit been modified in the past since their creation  

In [91]:
for file in repo1.git.diff('--name-only', fixing_commit_1, fixing_commit_1 + '^').splitlines():
    res9 = repo1.git.log('--follow', '--pretty=oneline', file).splitlines();
    print("  %s has been modified %d times"%(file, len(res9)))

  java/org/apache/catalina/core/ApplicationContext.java has been modified 71 times
  java/org/apache/catalina/servlets/WebdavServlet.java has been modified 44 times
  java/org/apache/catalina/ssi/SSIServletExternalResolver.java has been modified 20 times
  java/org/apache/catalina/ssi/SSIServletRequestUtil.java has been modified 9 times
  java/org/apache/catalina/util/RequestUtil.java has been modified 25 times
  java/org/apache/naming/resources/FileDirContext.java has been modified 40 times
  java/org/apache/tomcat/util/http/RequestUtil.java has been modified 1 times
  res/checkstyle/org-import-control.xml has been modified 8 times
  test/org/apache/catalina/util/TestRequestUtil.java has been modified 9 times
  test/org/apache/tomcat/util/http/TestRequestUtil.java has been modified 9 times


## Q(j) Which developers have modified each affected file since its creation?

In [92]:
for file in repo1.git.diff('--name-only',fixing_commit_1, fixing_commit_1 + '^').splitlines():
    print("  %s has been modified by following developers:"%file)
    res10 = repo1.git.log('--follow', '--pretty=%aN', file).splitlines()
    for dev in set(res10):
        print("    %s"%dev)

  java/org/apache/catalina/core/ApplicationContext.java has been modified by following developers:
    Remy Maucherat
    Mark Thomas
    Konstantin Kolinko
    Filip Hanik
    Mladen Turk
    Rainer Jung
  java/org/apache/catalina/servlets/WebdavServlet.java has been modified by following developers:
    Tim Whittington
    Remy Maucherat
    Mark Thomas
    Konstantin Kolinko
    Mladen Turk
    Rainer Jung
  java/org/apache/catalina/ssi/SSIServletExternalResolver.java has been modified by following developers:
    Konstantin Kolinko
    Mark Thomas
    Remy Maucherat
    Mladen Turk
  java/org/apache/catalina/ssi/SSIServletRequestUtil.java has been modified by following developers:
    Mark Thomas
    Remy Maucherat
    Mladen Turk
  java/org/apache/catalina/util/RequestUtil.java has been modified by following developers:
    Remy Maucherat
    Mark Thomas
    Konstantin Kolinko
    Filip Hanik
    Mladen Turk
    Rainer Jung
  java/org/apache/naming/resources/FileDirContext.java ha

## Q(k) For each developer identified, how many commits have each of them submitted? From your observation, are the involving developers experienced (with many commits) or new ones (with few commits) or both? 

In [93]:
devs = []
for file in repo1.git.diff('--name-only',fixing_commit_1, fixing_commit_1 + '^').splitlines():
    devs += repo1.git.log('--follow', '--pretty=%aN', file).splitlines()
res11 = list(set(devs))
log = repo1.git.log('--pretty=%aN')
for dev in res11:
    print(" %s made %d commits"%(dev, len(re.findall(dev,log))))

 Tim Whittington made 22 commits
 Remy Maucherat made 434 commits
 Timothy A. Funk made 61 commits
 Mark Thomas made 4729 commits
 Konstantin Kolinko made 675 commits
 Filip Hanik made 1000 commits
 Mladen Turk made 124 commits
 Rainer Jung made 280 commits


## Case 2 CVE-2018-1325

In [94]:
repo2 = Repo("../Documents/wicket-jquery-ui")
fixing_commit_2 = "a7b9f7961e"

## Q(a) What was the message and title of the fixing commit? Was there any mention of fixing a bug or vulnerability? 

In [95]:
res1 = repo2.git.log(-1, '--pretty=%B', fixing_commit_2)

print("Title:",res1)

Title: Added WysiwygEditor



## Q(b) How many total files were affected in the fixing commit? 

In [96]:
res2 = repo2.git.diff('--name-only', fixing_commit_2, fixing_commit_2 + '^').splitlines()
print("Number of affected files: %d"%len(res2))

Number of affected files: 25


## Q(c) How many total directories were affected in the fixing commit? For example, if a file path is: abc/def/File.java, then its directory is abc/def.

In [97]:
res3 = repo2.git.diff('--dirstat', fixing_commit_2, fixing_commit_2 + '^').splitlines()
print("Number of affected directories: %d"%len(res3))

Number of affected directories: 4


## Q(d) How many total lines of code (including comments and blank lines) were deleted? 

In [98]:
res4 = repo2.git.show(fixing_commit_2).splitlines()
res4 = [line for line in res4 if re.compile('^-$|^-[^-]').match(line)]
print("Number of deleted lines (including comments and blank lines): %d"%len(res4))

Number of deleted lines (including comments and blank lines): 2


## Q(e) How many total lines of code (including comments and blank lines) were added?

In [99]:
res5 = repo2.git.show(fixing_commit_2).splitlines()
res5 = [line for line in res5 if re.compile('^\+$|^\+[^\+]').match(line)]
print("Number of added lines (including comments and blank lines): %d"%len(res5))

Number of added lines (including comments and blank lines): 4478


## Q(f) How many total lines of code (excluding comments and blank lines) were deleted? 

In [100]:
res6 = repo2.git.show('--ignore-blank-lines', fixing_commit_2).splitlines()
res6 = [line for line in res6 if re.compile('^-[^-]').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*/').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res6))

Number of added lines (excluding comments and blank lines): 0


## Q(g) How many total lines of code (excluding comments and blank lines) were added? 

In [101]:
res7 = repo2.git.show('--ignore-blank-lines', fixing_commit_2).splitlines()
res7 = [line for line in res7 if re.compile('^\+[^\+]').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*/').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res7))

Number of added lines (excluding comments and blank lines): 3515


## Q(h) How many days were between the current fixing commit and the previous commit of each affected file?

In [102]:
for file in repo2.git.diff('--name-only', fixing_commit_2, fixing_commit_2 + '^').splitlines():
    #print(file)
    res8 = repo2.git.log(-2, '--pretty=%ct', file).splitlines();
    if len(res8) == 2:
        print(" For %s , the days between the current fixing commit and the previous commit is %.1f days"%(file, (int(res8[0])-int(res8[1]))/86400))
    else:
        print(" %s:  N/A This file doesn't have previous commit"%file)

 wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygBehavior.java:  N/A This file doesn't have previous commit
 wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.html:  N/A This file doesn't have previous commit
 wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.java:  N/A This file doesn't have previous commit
 wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/toolbar/DefaultWysiwygToolbar.html:  N/A This file doesn't have previous commit
 wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/toolbar/DefaultWysiwygToolbar.java:  N/A This file doesn't have previous commit
 wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/toolbar/IWysiwygToolbar.java:  N/A This file doesn't have previous commit
 wicket-jquery-ui-plugins/src/main/resources

## Q(i) How many time has each affected file of the current fixing commit been modified in the past since their creation  

In [103]:
for file in repo2.git.diff('--name-only', fixing_commit_2, fixing_commit_2 + '^').splitlines():
    res9 = repo2.git.log('--follow', '--pretty=oneline', file).splitlines();
    print("  %s has been modified %d times"%(file, len(res9)))

  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygBehavior.java has been modified 1 times
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.html has been modified 4 times
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.java has been modified 1 times
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/toolbar/DefaultWysiwygToolbar.html has been modified 1 times
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/toolbar/DefaultWysiwygToolbar.java has been modified 5 times
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/toolbar/IWysiwygToolbar.java has been modified 7 times
  wicket-jquery-ui-plugins/src/main/resources/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/css/bootstrap-combined.no-icons.min.css has been modified 

## Q(j) Which developers have modified each affected file since its creation?

In [104]:
for file in repo2.git.diff('--name-only',fixing_commit_2, fixing_commit_2 + '^').splitlines():
    print("  %s has been modified by following developers:"%file)
    res10 = repo2.git.log('--follow', '--pretty=%aN', file).splitlines()
    for dev in set(res10):
        print("    %s"%dev)

  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygBehavior.java has been modified by following developers:
    Sebastien
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.html has been modified by following developers:
    Sebastien
    Sebastien Briquet
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.java has been modified by following developers:
    Sebastien
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/toolbar/DefaultWysiwygToolbar.html has been modified by following developers:
    Sebastien
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/toolbar/DefaultWysiwygToolbar.java has been modified by following developers:
    Sebastien
    Sebastien Briquet
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/toolbar/

## Q(k) For each developer identified, how many commits have each of them submitted? From your observation, are the involving developers experienced (with many commits) or new ones (with few commits) or both? 

In [105]:
devs = []
for file in repo2.git.diff('--name-only',fixing_commit_2, fixing_commit_2 + '^').splitlines():
    devs += repo2.git.log('--follow', '--pretty=%aN', file).splitlines()
res11 = list(set(devs))
log = repo2.git.log('--pretty=%aN')
for dev in res11:
    print(" %s made %d commits"%(dev, len(re.findall(dev,log))))

 Sebastien made 186 commits
 Sebastien Briquet made 175 commits


## Case 3 CVE-2013-7251

In [106]:
repo3 = Repo("~/Documents/projectforge-webapp")
fixing_commit_3 = "96ca7e5ac0"

## Q(a) What was the message and title of the fixing commit? Was there any mention of fixing a bug or vulnerability? 

In [107]:
res1 = repo3.git.log(-1, '--pretty=%B', fixing_commit_3)

print("Title:",res1)

Title: PF-396: Disable autocompletion in all forms by default.

git-svn-id: https://svn.micromata.de/svn/projectforge/webapp/trunk@14054 be79319d-3443-0410-9b0a-b0c1b946f33b



## Q(b) How many total files were affected in the fixing commit? 

In [108]:
res3 = repo3.git.diff('--name-only', fixing_commit_3, fixing_commit_3 + '^').splitlines()
print("Number of affected files: %d"%len(res3))

Number of affected files: 4


## Q(c) How many total directories were affected in the fixing commit? For example, if a file path is: abc/def/File.java, then its directory is abc/def.

In [109]:
res3 = repo3.git.diff('--dirstat', fixing_commit_3, fixing_commit_3 + '^').splitlines()
print("Number of affected directories: %d"%len(res3))

Number of affected directories: 3


## Q(d) How many total lines of code (including comments and blank lines) were deleted? 

In [110]:
res4 = repo3.git.show(fixing_commit_3).splitlines()
res4 = [line for line in res4 if re.compile('^-$|^-[^-]').match(line)]
print("Number of deleted lines (including comments and blank lines): %d"%len(res4))

Number of deleted lines (including comments and blank lines): 4


## Q(e) How many total lines of code (including comments and blank lines) were added?

In [111]:
res5 = repo3.git.show(fixing_commit_3).splitlines()
res5 = [line for line in res5 if re.compile('^\+$|^\+[^\+]').match(line)]
print("Number of added lines (including comments and blank lines): %d"%len(res5))

Number of added lines (including comments and blank lines): 4


## Q(f) How many total lines of code (excluding comments and blank lines) were deleted? 

In [112]:
res6 = repo3.git.show('--ignore-blank-lines', fixing_commit_3).splitlines()
res6 = [line for line in res6 if re.compile('^-[^-]').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*/').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res6))

Number of added lines (excluding comments and blank lines): 4


## Q(g) How many total lines of code (excluding comments and blank lines) were added? 

In [113]:
res7 = repo3.git.show('--ignore-blank-lines', fixing_commit_3).splitlines()
res7 = [line for line in res7 if re.compile('^\+[^\+]').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*/').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res7))

Number of added lines (excluding comments and blank lines): 4


## Q(h) How many days were between the current fixing commit and the previous commit of each affected file?

In [114]:
for file in repo3.git.diff('--name-only', fixing_commit_3, fixing_commit_3 + '^').splitlines():
    res8 = repo3.git.log(-2, '--pretty=%ct', file).splitlines();
    if len(res8) == 2:
        print(" For %s , the days between the current fixing commit and the previous commit is %.1f days"%(file, (int(res8[0])-int(res8[1]))/86400))
    else:
        print(" %s:  N/A This file doesn't have previous commit"%file)

 For src/main/java/org/projectforge/web/dialog/ModalDialog.html , the days between the current fixing commit and the previous commit is 2.0 days
 For src/main/java/org/projectforge/web/fibu/RechnungCostEditTablePanel.html , the days between the current fixing commit and the previous commit is 36.0 days
 For src/main/java/org/projectforge/web/mobile/AbstractMobileEditPage.html , the days between the current fixing commit and the previous commit is 36.0 days
 For src/main/java/org/projectforge/web/mobile/AbstractMobileListPage.html , the days between the current fixing commit and the previous commit is 36.0 days


## Q(i) How many time has each affected file of the current fixing commit been modified in the past since their creation  

In [115]:
for file in repo3.git.diff('--name-only', fixing_commit_3, fixing_commit_3 + '^').splitlines():
    res9 = repo3.git.log('--follow', '--pretty=oneline', file).splitlines();
    print("  %s has been modified %d times"%(file, len(res9)))

  src/main/java/org/projectforge/web/dialog/ModalDialog.html has been modified 8 times
  src/main/java/org/projectforge/web/fibu/RechnungCostEditTablePanel.html has been modified 3 times
  src/main/java/org/projectforge/web/mobile/AbstractMobileEditPage.html has been modified 11 times
  src/main/java/org/projectforge/web/mobile/AbstractMobileListPage.html has been modified 12 times


## Q(j) Which developers have modified each affected file since its creation?

In [116]:
for file in repo3.git.diff('--name-only',fixing_commit_3, fixing_commit_3 + '^').splitlines():
    print("  %s has been modified by following developers:"%file)
    res10 = repo3.git.log('--follow', '--pretty=%aN', file).splitlines()
    for dev in set(res10):
        print("    %s"%dev)

  src/main/java/org/projectforge/web/dialog/ModalDialog.html has been modified by following developers:
    kai
  src/main/java/org/projectforge/web/fibu/RechnungCostEditTablePanel.html has been modified by following developers:
    kai
  src/main/java/org/projectforge/web/mobile/AbstractMobileEditPage.html has been modified by following developers:
    kai
  src/main/java/org/projectforge/web/mobile/AbstractMobileListPage.html has been modified by following developers:
    kai


## Q(k) For each developer identified, how many commits have each of them submitted? From your observation, are the involving developers experienced (with many commits) or new ones (with few commits) or both? 

In [117]:
devs = []
for file in repo3.git.diff('--name-only',fixing_commit_3, fixing_commit_3 + '^').splitlines():
    devs += repo3.git.log('--follow', '--pretty=%aN', file).splitlines()
res11 = list(set(devs))
log = repo3.git.log('--pretty=%aN')
for dev in res11:
    print(" %s made %d commits"%(dev, len(re.findall(dev,log))))

 kai made 4013 commits
